In [1]:
include("../ToolBox/ToolBox.jl")
import JLD
import FileIO
using .ToolBox
using .Geometry
using Colors

In [2]:
dem_path = "/Users/simon/Data/DEM/denmark.jld";

In [5]:
function preprocess3_v1(product_folders, master_view, dem_path, subswath, result_folder, kernel = ones(4,14))
    
    n_coherence = length(product_folders)-1
    dem = 0
    lut1 = 0
    lut2 = 0
    lut_status = [false,false]
    precise_orbit = 0
    master_data = 0
    lines = 0
    samples = 0
    
    stride_line = floor(Int,size(kernel)[1]/2)
    stride_sample = floor(Int,size(kernel)[2]/2)
    
    pod_paths = [Load.pod_path(folder) for folder in product_folders]
    
    for polarization in ["VV","VH"]
        
        println(polarization)
        file_paths = [Load.slc_paths(folder, polarization, subswath) for folder in product_folders]
        meta = [Load.slc_meta(path[2]) for path in file_paths]
        calibration = [Load.slc_calibration(file_paths[i][3],meta[i]["t_0"]) 
                                    for i in 1:length(product_folders)];
        # load pod
        if precise_orbit == 0
            println("load POD")
            precise_orbit = [Load.precise_orbit(pod_paths[i],meta[i]["t_0"]) 
                                    for i in 1:length(product_folders)]
        end
        
        # load DEM
        if dem == 0
            println("load DEM")
            pad = 0.1
            dem = JLD.load(dem_path);
            footprint = SlcUtil.footprint(meta[2], master_view)
            latlon_window = ((minimum(footprint[1]), maximum(footprint[1])), (minimum(footprint[2]), maximum(footprint[2])))
            idx1 =(latlon_window[1][1] - pad) .<dem["lat"].< (latlon_window[1][2] + pad)
            idx2 =(latlon_window[2][1] - pad) .<dem["lon"].< (latlon_window[2][2] + pad)
            dem = (dem["lat"][idx1], dem["lon"][idx2], dem["height"][idx1,idx2]);
            dem[3][dem[3].<0] .= 0; 
        end
        
        for i in 1:n_coherence
            println("i: ",i)
            if i == 1
                lut = lut1
                index = [2,1]
            else
                lut = lut2
                index = [2,3]
            end
            
            # coreg_slave data
            if !(lut_status[i])
                println("coreg no lut")
                slave_data,flat,lut =  coregister_slave(master_view,file_paths[index[2]][1],
                                                        meta[index],precise_orbit[index],dem)
                # save lut
                if i==1
                    lut1 = lut
                else
                    lut2 = lut
                end
                lut_status[i] = true
                lut_path = joinpath(result_folder,string(index[1])*string(index[2])*"_lut.jld")
                JLD.save(lut_path, "data", lut)
                
            else
                println("coreg with lut")
                slave_data,flat =  coregister_slave(master_view,file_paths[index[2]][1],
                                                        meta[index],precise_orbit[index],dem,lut)
            end

            slave_data,mosaic_view = SlcUtil.mosaic(slave_data,master_view,meta[index[1]]);
            flat,mosaic_view = SlcUtil.mosaic(flat,master_view,meta[index[1]]);
            slave_data = SlcUtil.calibrate_slave_data(slave_data, mosaic_view,lut1, calibration[index[2]]);
            
            if i == 1
            # Get master data
                master_data = Load.slc_data(file_paths[index[1]][1],master_view);
                master_data,mosaic_view = SlcUtil.mosaic(master_data,master_view,meta[index[1]]);
                master_data = SlcUtil.calibrate_data(master_data, Misc.flatten(mosaic_view...)..., calibration[index[1]]);
            end

            # Compute coherence
            complex_coherence, master_intensity, slave_intensity, lines, samples = SlcUtil.complex_coherence(
                                                                master_data, slave_data, flat, kernel, mosaic_view);

            slave_data = 0

            # subsample 
            complex_coherence = complex_coherence[1:stride_line:end,1:stride_sample:end]
            slave_intensity = slave_intensity[1:stride_line:end,1:stride_sample:end]
            
            # save 
            slave_path = joinpath(result_folder,string(index[2])*polarization*"_itens.jld")
            master_path = joinpath(result_folder,string(index[1])*polarization*"_itens.jld")
            coherence_path = joinpath(result_folder,string(index[1])*string(index[2])*
                                                            polarization*"_coher.jld")
            println("Save")
            JLD.save(slave_path, "data", slave_intensity)
            JLD.save(coherence_path, "data", complex_coherence)
            if i == 1
                println("Save master intens")
                master_intensity = master_intensity[1:stride_line:end,1:stride_sample:end]
                JLD.save(master_path, "data", master_intensity)
            end
            
        end
        
    end
    # save lines and samples 
    line_sample_path = joinpath(result_folder,"line_sample_path.jld")
    JLD.save(line_sample_path, "lines", lines, "samples", samples)
    
    return 1
    
end

preprocess3_v1 (generic function with 2 methods)

In [ ]:
co_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20191011T170932_20191011T170959_029414_035865_06AB.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190929T170932_20190929T170959_029239_035263_C3F9.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190917T170931_20190917T170958_029064_034C5E_2484.SAFE"

result_folder = "/Users/simon/Data/Sentinel/vejle_oktober/factory_AAA"

master_A_view = [8000:10000,1200:6000];
product_folders = [co_folder,pre1_folder,pre2_folder];

preprocess3_v1(product_folders,master_A_view,dem_path,3, result_folder)

In [6]:
co_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20191011T170932_20191011T170959_029414_035865_06AB.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1B_IW_SLC__1SDV_20191005T170849_20191005T170916_018343_0228DC_F565.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190929T170932_20190929T170959_029239_035263_C3F9.SAFE"

result_folder = "/Users/simon/Data/Sentinel/vejle_oktober/factory_ABA"

product_folders = [co_folder,pre1_folder,pre2_folder];

master_B_view = [9500:11500,1200:6000];

preprocess3_v1(product_folders,master_B_view,dem_path,3, result_folder)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save


1

In [38]:
subswath = 3
kernel = ones(4,14)
n_coherence = length(product_folders)-1
dem = 0
lut1 = 0
lut2 = 0
lut_status = [false,false]
precise_orbit = 0
master_data = 0
lines = 0
samples = 0

stride_line = floor(Int,size(kernel)[1]/2)
stride_sample = floor(Int,size(kernel)[2]/2)

pod_paths = [Load.pod_path(folder) for folder in product_folders];

polarization = "VV"

 println(polarization)
        file_paths = [Load.slc_paths(folder, polarization, subswath) for folder in product_folders]
        meta = [Load.slc_meta(path[2]) for path in file_paths]
        calibration = [Load.slc_calibration(file_paths[i][3],meta[i]["t_0"]) 
                                    for i in 1:length(product_folders)];
        # load pod
        if precise_orbit == 0
            println("load POD")
            precise_orbit = [Load.precise_orbit(pod_paths[i],meta[i]["t_0"]) 
                                    for i in 1:length(product_folders)]
        end
        
        # load DEM
        if dem == 0
            println("load DEM")
            pad = 0.1
            dem = JLD.load(dem_path);
            footprint = SlcUtil.footprint(meta[1], master_view)
            latlon_window = ((minimum(footprint[1]), maximum(footprint[1])), (minimum(footprint[2]), maximum(footprint[2])))
            idx1 =(latlon_window[1][1] - pad) .<dem["lat"].< (latlon_window[1][2] + pad)
            idx2 =(latlon_window[2][1] - pad) .<dem["lon"].< (latlon_window[2][2] + pad)
            dem = (dem["lat"][idx1], dem["lon"][idx2], dem["height"][idx1,idx2]);
            dem[3][dem[3].<0] .= 0; 
        end



VV
load POD
load DEM


24083-element view(::Array{Int16,1}, [124020, 124550, 125080, 151331, 151332, 151858, 151859, 151860, 151861, 151862  …  335949, 335950, 335951, 335952, 335953, 335954, 335955, 335956, 335957, 335958]) with eltype Int16:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [39]:
i = 1
println("i: ",i)
            if i == 1
                lut = lut1
                index = [2,1]
            else
                lut = lut2
                index = [2,3]
            end
            

i: 1


2-element Array{Int64,1}:
 2
 1

In [64]:
meta = meta[index]
precise_orbit[index]
slave_data_path = file_paths[1][1]
master_view = [9500:11500,1200:1400]

2-element Array{UnitRange{Int64},1}:
 9500:11500
 1200:6000 

In [65]:
mosaic_view = SlcUtil.mosaic_view(meta[1],master_view)
lut = look_up_table(mosaic_view,meta,precise_orbit,dem);

In [73]:
minimum(lut["slave_line"]),minimum(lut["slave_sample"]),minimum(lut["heights"])

(9807.114880273606, NaN, NaN)

In [74]:
minimum(lut["latitude"]),minimum(lut["longitude"])

(NaN, NaN)

In [81]:
sum(isnan.(lut["master_line"]))

0

In [ ]:
lut["slave_sample"]

In [66]:
    c = 299792458
    range_pixel_spacing =  c/(2*meta[1]["range_sampling_rate"])
    lambda =  c/meta[1]["radar_frequency"]

    # Get the midburst speed
    v_mid = mid_burst_speed(precise_orbit[2], meta[2]);

    # get master line and sample
    mosaic_view = SlcUtil.mosaic_view(meta[1],master_view)
    master_line, master_sample = Misc.flatten(mosaic_view...)

    # interpolate slave line and sample
    slave_line = Misc.interp_grid(lut["master_line"] ,lut["master_sample"],
    reshape(lut["slave_line"],(length(lut["master_line"]),length(lut["master_sample"])))
    ,mosaic_view[1], mosaic_view[2])
    slave_sample = Misc.interp_grid(lut["master_line"] ,lut["master_sample"],
        reshape(lut["slave_sample"],(length(lut["master_line"]),length(lut["master_sample"])))
        ,mosaic_view[1], mosaic_view[2]);

In [67]:
minimum(slave_line),minimum(slave_sample)

(9807.114880273606, NaN)

In [52]:
slave_line = Misc.interp_grid(lut["master_line"] ,lut["master_sample"],
    reshape(lut["slave_line"],(length(lut["master_line"]),length(lut["master_sample"])))
    ,mosaic_view[1], mosaic_view[2])
    slave_sample = Misc.interp_grid(lut["master_line"] ,lut["master_sample"],
        reshape(lut["slave_sample"],(length(lut["master_line"]),length(lut["master_sample"])))
        ,mosaic_view[1], mosaic_view[2]);

In [56]:
slave_line = reshape(slave_line,:)
    slave_sample= reshape(slave_sample,:);

    # master bursts
    master_start_burst = ceil(Int,(master_view[1].start)/meta[1]["lines_per_burst"])
    master_end_burst = ceil(Int,(master_view[1].stop-1)/meta[1]["lines_per_burst"])

    # find the slave bursts
    slave_start_line = minimum(slave_line)
    slave_start_burst = sum([slave_start_line > elem ? 1 : 0 for elem in meta[2]["burst_meta"]["first_line_mosaic"]])

    ## Get the number of lines in the first selected burst
    lines_in_first_burst_master = master_start_burst*meta[1]["lines_per_burst"] - master_view[1].start
    lines_in_first_burst_slave = meta[2]["burst_meta"]["first_line_mosaic"][slave_start_burst]+meta[2]["lines_per_burst"] - slave_start_line

    ## Check if the number of lines are approximately the same
    if lines_in_first_burst_slave > lines_in_first_burst_master + 100
        # the number of lines are not the same. The slave burst before should be included
        slave_start_burst = slave_start_burst -1
        lines_in_first_burst_slave = slave_start_burst*meta[2]["lines_per_burst"] - slave_start_line
    end

    # compute the offset between the burst number in slave and master
    delta_burst = slave_start_burst - master_start_burst

    ## check if it is in slave image
    slave_end_burst = master_end_burst + delta_burst

    if slave_start_burst == 0
        println("Warning: start line not in slave image")
    end

    if slave_end_burst > meta[2]["burst_count"]
        println("Warning: end line not in slave image")
    end

    if minimum(slave_sample) < 1
        println("Warning: start sample not in slave image")
    end

    if maximum(slave_sample) > meta[2]["samples_per_burst"]
        println("Warning: end sample not in slave image")
    end

    # Initialize arrays for the results
    coreg_slave = Array{Complex{Float64}}(undef,length.(master_view)...)
    flat_inferogram = Array{Complex{Float64}}(undef,length.(master_view)...);


2001×4801 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
    ⋮                             ⋱                           ⋮   

In [60]:
slave_start_line = minimum(slave_line)

7097.885622660336

In [61]:
slave_start_burst = sum([slave_start_line > elem ? 1 : 0 for elem in meta[2]["burst_meta"]["first_line_mosaic"]])

6

In [62]:
ceil(Int,(master_view[1].start)/meta[1]["lines_per_burst"])

6

In [63]:
master_view[1]

8000:10000

In [55]:
minimum(slave_line),minimum(slave_sample)

(7097.885622660336, 1533.4104282626326)